![ngo_project_image](ngo_project_image.jpg)

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="50%" height="50%">


You will execute SQL queries to answer two questions, as listed in the instructions. Good luck!


In [69]:
-- highest_donation_assignments
SELECT a.assignment_name,
       a.region,
       ROUND(SUM(d.amount), 2) AS rounded_total_donation_amount,
       d1.donor_type,
       TO_CHAR(d.donation_date, 'YYYY-MM-DD') AS date,
	   a.impact_score
FROM assignments as a
INNER JOIN donations as d
ON a.assignment_id = d.assignment_id 
INNER JOIN donors as d1
ON d.donor_id = d1.donor_id
GROUP BY a.assignment_name, a.region, d1.donor_type, date, a.impact_score
ORDER BY rounded_total_donation_amount DESC;

,assignment_name,region,rounded_total_donation_amount,donor_type,date,impact_score
0,Assignment_192,North,999.90,Organization,2020-06-06,2.54
1,Assignment_1899,North,999.84,Individual,2022-08-24,1.97
2,Assignment_3277,East,999.82,Corporate,2022-08-12,1.13
3,Assignment_1874,West,999.32,Organization,2022-11-12,9.76
4,Assignment_4156,South,999.13,Organization,2020-06-26,2.82
...,...,...,...,...,...,...
4995,Assignment_3738,South,10.94,Corporate,2020-03-24,9.57
4996,Assignment_1828,South,10.84,Organization,2021-05-29,7.31
4997,Assignment_4958,South,10.65,Individual,2020-07-03,4.41
4998,Assignment_4827,South,10.62,Organization,2021-02-10,8.92


In [50]:
-- top_regional_impact_assignments
WITH d1 AS (
    SELECT a.assignment_name,
           a.region,
           a.impact_score,
           COUNT(d.donation_id) AS total_donation
    FROM assignments AS a
    INNER JOIN donations AS d
    ON a.assignment_id = d.assignment_id 
    GROUP BY a.assignment_name,
             a.region,
             a.impact_score
),
ranked_assignments AS (
    SELECT assignment_name,
           region,
           impact_score,
           total_donation,
           ROW_NUMBER() OVER(PARTITION BY region ORDER BY impact_score DESC) as rank
    FROM d1
)
SELECT assignment_name,
       region,
       impact_score,
       total_donation as num_total_donations
FROM ranked_assignments
WHERE rank = 1
ORDER BY region;

,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_2794,West,9.99,2


In [51]:
SELECT *
FROM assignments
LIMIT 5;

,assignment_id,assignment_name,start_date,end_date,budget,region,impact_score
0,1,Assignment_1,2021-10-17,2021-12-04,-32322.03,West,5.55
1,2,Assignment_2,2020-10-26,2020-11-28,57278.40,South,1.45
2,3,Assignment_3,2021-08-11,2022-03-17,40414.51,West,2.34
3,4,Assignment_4,2021-11-22,2022-05-17,31732.48,East,7.05
4,5,Assignment_5,2020-11-22,2021-07-10,13548.22,North,5.29


In [52]:
SELECT *
FROM donations
LIMIT 5;

,donation_id,donor_id,amount,donation_date,assignment_id
0,1,2733,271.36,2021-08-21 00:00:00+00:00,4226
1,2,2608,251.49,2021-10-15 00:00:00+00:00,1323
2,3,1654,528.38,2020-03-03 00:00:00+00:00,4881
3,4,3265,730.36,2021-02-06 00:00:00+00:00,1237
4,5,4932,285.96,2022-03-05 00:00:00+00:00,1626


In [53]:
SELECT *
FROM donors
LIMIT 5;

,donor_id,donor_name,donor_type
0,1,Donor_1,Individual
1,2,Donor_2,Organization
2,3,Donor_3,Individual
3,4,Donor_4,Organization
4,5,Donor_5,Organization
